In [1]:
import re
import glob
import csv
from helpers import config
from helpers.loading import load_daily_data ,file_exist

C:\Big Data\exchange-price-propagation\helpers\config.yml


In [66]:
signal  = "trade"
stock = "MFST"
def get_all_dates(stock,signal):
    """return a sorted list of all dates were trades/bbo (signal) are available in the data"""
    
    def extract_date(s):
        try:
            date = re.search(r"[0-9]{4}-[0-9]{2}-[0-9]{2}",s).group(0)
        except :
            print(s)
        return date
    all_files = glob.glob(f"./Data/{signal}/*/*")
    all_dates = [extract_date(s) for s in all_files] 
    all_dates = list(set(all_dates))
    all_dates.sort()
    return all_dates

In [68]:
all_dates = get_all_dates(stock,signal)

In [ ]:
fieldnames = ['date', 'market1','market2',"lag"]
write_header = not file_exist(config["files"]["results"]["all_best_lags"]) # need to write the header only if the file does not already exists
csvfile = open(config["files"]["results"]["all_best_lags"], 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
if write_header:
    writer.writeheader()
    
for date in all_dates: 
    best_lags = {}
    daily_data = load_daily_data(date)
    for i,n1 in enumerate(daily_data):
        for j,n2 in enumerate(daily_data):
            if i>j: # avoid symetric (corr(a,b)=corr(b,a)) and meaningless (corr(a,a)=1) calculations
                best_lag = find_best_delay_demo(n1,n2,step_size = 1000)
                # write the computed result
                writer.writerow({'date': '2010', 'market1': n1,'market2': n2,'lag': 10})
                
    csvfile.flush() # flush every time we processed a date
                
    break

csvfile.close()

In [34]:
fieldnames = ['date', 'market1','market2',"lag"]
write_header = not file_exist(config["files"]["results"]["all_best_lags"])
csvfile = open(config["files"]["results"]["all_best_lags"], 'a', newline='') 
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
if write_header:
    writer.writeheader()
writer.writerow({'date': '2010', 'market1': 'Beans','market2': 'Beans','lag': 10})
                
csvfile.flush() # flush every time we processed a date

In [35]:
csvfile.close()

In [ ]:

    
    

    
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})